In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Setting seed to make deterministic network
np.random.seed(42)
tf.random.set_seed(42)
tf.keras.utils.set_random_seed(42)  # sets seeds for base-python, numpy and tf
tf.config.experimental.enable_op_determinism()

2024-02-12 23:54:19.837075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load and preprocess data
train_df = pd.read_csv("train.csv")
print(train_df.head())
#Encoding categorical feature in train set
train_df.X10=train_df.X10.astype('category').cat.codes

train_features = train_df.drop("Y", axis=1).values
train_label = train_df["Y"].values

          X1         X2         X3         X4          X5          X6  \
0  20.236432  67.523185  35.766385  68.459483   74.946516   89.632851   
1  10.551182  57.675655  51.525733  49.891951  113.074296   81.223638   
2  14.069105  33.115667  60.014587  48.412263   88.815278  128.651604   
3  15.537824  28.032600  68.797017  55.482058   59.269249  103.644283   
4  10.791858  46.429463  48.373435  41.870487  101.306254  119.684299   

          X7          X8          X9  X10           Y  
0  78.277026  116.827922  122.975621  DEB  842.547441  
1  74.534979   92.063753  114.186779  AED  842.138909  
2  79.616572  145.231095  122.473611  DAF  820.804741  
3  77.766831  135.170297  145.037862  ACV  962.565837  
4  75.929410  103.408770  140.392891  DAF  758.052022  


In [3]:
#Splitting the train dataset in 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(train_features, train_label, test_size=0.2, random_state=42)

#Using Standard scaler to normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [4]:
# Defined the checkpoint filepath
checkpoint_filepath = 'model_checkpoint.ckpt'

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

In [5]:
def model_init():
    #Initializing the network
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(1, X_train_scaled.shape[1])))
    # Adding Batch Normalization for normalizing the inputs of each layer
    model.add(BatchNormalization())
    model.add(Dense(units=70, activation='relu'))
    # Adding Batch Normalization for normalizing the inputs of each layer
    model.add(BatchNormalization())
    model.add(Dense(units=70, activation='relu'))
    # Adding Batch Normalization for normalizing the inputs of each layer
    model.add(BatchNormalization())
    model.add(Dense(units=70, activation='relu'))
    # Adding Batch Normalization for normalizing the inputs of each layer
    model.add(BatchNormalization())
    model.add(Dense(units=1))

    #Defining the optimizer
    opt = SGD(learning_rate=0.0005, momentum=0.45, nesterov=True)
    model.compile(optimizer=opt, loss='mean_absolute_error')
    model.summary()
    
    return model

In [6]:
train_model = model_init()

print("\nBeginning model training\n")

# Train the model
history = train_model.fit(x=X_train_reshaped, y=y_train, epochs=400, batch_size=64, 
                    validation_data=(X_test_reshaped, y_test), 
                    callbacks=[checkpoint, EarlyStopping(monitor='val_loss', patience=10)])

print("Train MAE: ",train_model.evaluate(X_train_reshaped, y_train))
print("Validation MAE: ",train_model.evaluate(X_test_reshaped, y_test))

print("\nTraining complete\n")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 batch_normalization (Batch  (None, 50)                200       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 70)                3570      
                                                                 
 batch_normalization_1 (Bat  (None, 70)                280       
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 batch_normalization_2 (Bat  (None, 70)                2

In [7]:
#Initializing model for prediction
pred_model = model_init()
try:
    #Loading model weights from checkpoint stored in disk
    pred_model.load_weights(checkpoint_filepath)
    print("Model weights loaded in memory successfully.")
except Exception as e:
    print("Model checkpoint not found. Please train the model again.")
    print(e)
    exit(0)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                12200     
                                                                 
 batch_normalization_4 (Bat  (None, 50)                200       
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 70)                3570      
                                                                 
 batch_normalization_5 (Bat  (None, 70)                280       
 chNormalization)                                                
                                                                 
 dense_5 (Dense)             (None, 70)                4970      
                                                                 
 batch_normalization_6 (Bat  (None, 70)               

In [8]:
#Reading unseen data from Disk
test_df = pd.read_csv("test.csv")
#Encoding categorical feature in test dataset
test_df.X10=test_df.X10.astype('category').cat.codes

actual = test_df.drop("Y", axis=1).values
actual_scaled = scaler.transform(actual)

# Reshape the input data for LSTM (samples, time steps, features)
actual_reshaped = np.reshape(actual_scaled, (actual_scaled.shape[0], 1, actual_scaled.shape[1]))

# Extract actual target values
y_actual = test_df["Y"].values

In [9]:
print("Beginning prediction")
predictions = pred_model.predict(actual_reshaped)

# Create a DataFrame for predictions
predictions_df = pd.DataFrame(predictions, columns=["Predictions"])
predictions_df.head()

Beginning prediction
32/32 [==============================] - 1s 3ms/step


,Predictions
0,865.791931
1,724.061890
2,905.079041
3,863.038391
4,776.875427


In [10]:
mae = metrics.mean_absolute_error(y_actual, predictions)
mse = metrics.mean_squared_error(y_actual, predictions)
rmse = np.sqrt(mse)

print("MAE on test data: ", mae)
print("MSE on test data: ", mse)
print("RMSE on test data:", rmse)

MAE on test data:  7.675269873148509
MSE on test data:  89.4312669323208
RMSE on test data: 9.456810610999925


In [11]:
#Reading test dataset from disk again to avoid any earlier changes to original data
test_df_2 = pd.read_csv("test.csv")
test_prediction_concat = pd.concat([test_df_2, predictions_df], axis=1)
test_prediction_concat.to_csv("test_pred.csv", index=False)